In [8]:
_predict_df = "select (Version.[Version Name]*Product.[Product].[196426]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource,PluginSetting

# register inputs
predict_df = O9DataLake.register("predict_df",data_source = DataSource.LS, entity_type = ResourceType.IBPL, query = _predict_df,plugin_setting = PluginSetting.Inputs)
systemreport = O9DataLake.register("sysreport",data_source = DataSource.LIVEFRAME,entity_type = ResourceType.LIVEFRAME,plugin_setting = PluginSetting.Inputs)

# register slice dimension
O9DataLake.register("Product.[Product]", data_source = DataSource.LS, entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.SliceDimension)

# register outputs
O9DataLake.register("output1",data_source = DataSource.LS,entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.Outputs)

# register script params
script_params = O9DataLake.register({"var1":"10","var2":"Test1"}, data_source = DataSource.LS, plugin_setting = PluginSetting.ScriptParam)

In [9]:
O9DataLake.inputs

{'predict_df': {'name': 'predict_df',
  'resource_type': <ResourceType.IBPL: 'ibpl_query'>,
  'data_source': <DataSource.LS: 'liveserver'>,
  'query': 'select (Version.[Version Name]*Product.[Product].[196426]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});',
  'std_count_limit': '200000',
  'df':    Version.[Version Name]  Product.[Product] Time.[FiscalWeek]  \
  0                      S1             196426          W03-2016   
  1      CurrentWorkingView             196426          W03-2016   
  2                      S1             196426          W05-2016   
  3      CurrentWorkingView             196426          W05-2016   
  4                      S1             196426          W08-2016   
  5      CurrentWorkingView             196426          W08-2016   
  6                      S1             196426          W13-2016   
  7

In [10]:
# fetching inputs
predict_df = O9DataLake.get('predict_df')
systemreport = O9DataLake.get('sysreport')

# fetching script params
value1 = O9DataLake.get_script_param("var1")
value2 = O9DataLake.get_script_param("var2")

In [11]:
#user script

# package imports
import logging
from sklearn import tree

# initialize output variables
output1 = None


# initialize logger
logger = logging.getLogger('o9_logger')

logger.debug(f'predict_df dataframe:  {predict_df.shape}')
logger.debug(f'systemreport dataframe:  {systemreport.shape}')
logger.debug(f'script param var1 value: {value1}')
logger.debug(f'script param var2 value: {value2}')

2024-05-02 12:38:47,638 - o9_logger - DEBUG  - predict_df dataframe:  (14, 10)
2024-05-02 12:38:47,639 - o9_logger - DEBUG  - systemreport dataframe:  (8190, 4)
2024-05-02 12:38:47,640 - o9_logger - DEBUG  - script param var1 value: 10
2024-05-02 12:38:47,641 - o9_logger - DEBUG  - script param var2 value: Test1


In [12]:
output1 = systemreport
logger.debug(f'systemreport dataframe:  {systemreport.head()}')

2024-05-02 12:38:47,653 - o9_logger - DEBUG  - systemreport dataframe:    WalmartTime.[Day]  Store.[Store_ID] Version.[Version Name]  DSML dsmldim
0        02-05-2010                 1     CurrentWorkingView         2.572
1        02-05-2010                10     CurrentWorkingView         3.963
2        02-05-2010                11     CurrentWorkingView         2.653
3        02-05-2010                12     CurrentWorkingView         4.121
4        02-05-2010                13     CurrentWorkingView         2.797


In [13]:
#pushing outputs to Live Server(make sure all dimensions,measures present in LiveServer)
O9DataLake.put('output1', output1)